In [149]:

#!/usr/bin/env python3

import argparse
import json
import os
import importlib.util

from src.learner.raytune_learner import TuneWrapper as StimulusTuneWrapper
from src.learner.raytune_learner import TuneModel as StimulusTuneModel
from launch_utils import import_class_from_file, get_experiment
from src.utils.yaml_model_schema import YamlRayConfigLoader
import torch 
import src.data.handlertorch as handlertorch
from src.learner.predict import PredictWrapper
import torch.nn as nn
import torch.optim as optim 


model_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/fpkm_dummy_model.py"
json_experiment = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/fpkm_pipelinegenerated.json"
config_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/fpkm_dummy_model.yaml"
data_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/test_basic.csv"
best_config_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/best_config.json"



# import the model correctly but do not initialize it yet, ray_tune does that itself
model_class = import_class_from_file(model_path)
model_class()
# read json and retrieve experiment name and then initialize the experiment class
experiment_name = None
with open(json_experiment, 'r') as in_json:
    d = json.load(in_json)
    experiment_name = d["experiment"]
experiment = get_experiment(experiment_name)


dataset = handlertorch.TorchDataset(os.path.abspath(data_path), experiment, split=None)
train_data = torch.utils.data.DataLoader(dataset, batch_size=1000, shuffle=True)


# Create the learner
learner = StimulusTuneWrapper(config_path, model_class, data_path, experiment)
# learner.tune()
# learner.store_best_config(best_config_path)cat


config_loader = YamlRayConfigLoader(config_path)
config = config_loader.config

loss_dict = {"loss_fn": "PoissonNLLLoss"}
for key, loss_fn in loss_dict.items():
    try:
        loss_dict[key] = getattr(nn, loss_fn)()
    except AttributeError:
        raise ValueError(f"Invalid loss function: {loss_fn}, check PyTorch for documentation on available loss functions")
from src.learner.predict import PredictWrapper

%load_ext autoreload
%autoreload 2

# Prepare the model and optimizer

config["model_params"]["input_length"] = 10 
config["model_params"]["nfilters_conv1"] = 4
config["model_params"]["kernel_size_1"] = 5
config["optimizer_params"]["lr"] = 0.01
config["optimizer_params"]["method"] = "Adam"

optimizer_lr = config["optimizer_params"]["lr"]
model = model_class(**config["model_params"])
optimizer = getattr(optim, config["optimizer_params"]["method"])(model.parameters(), lr=optimizer_lr)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [150]:
epochs = 2
for epoch in range(epochs):
    print(f"Epoch {epoch}")
    for i, (sequence, fpkm, specie, ) in enumerate(train_data):
        print(f"Batch {i}")
        model.batch(sequence, fpkm,  loss_dict["loss_fn"], optimizer)


Epoch 0
Batch 0
Optimizing
Loss: 2.2182817459106445
Epoch 1
Batch 0
Optimizing
Loss: 2.2182817459106445


In [151]:
df

,sequence:input:dna,fpkm:label:float,split:split:int,species:meta:str
0,CCCCTAAGTC,0,0,Homo_sapiens
1,CTATATACTA,1,0,Homo_sapiens
2,TATATAGGTA,1,0,Homo_sapiens
3,GAGTGTGACT,0,1,Homo_sapiens
4,ATGAGATGGC,0,0,Homo_sapiens
5,ACCCACAAGT,0,0,Homo_sapiens
6,GTATATATTC,1,0,Homo_sapiens
7,GTATATAGAG,1,2,Homo_sapiens
8,ACACAGTTAG,0,0,Homo_sapiens
9,TGGTATATAG,1,0,Homo_sapiens


In [108]:
# check how many positives there are per class
import pandas as pd
df = pd.read_csv(data_path)
df.head()

,sequence:input:dna,fpkm:label:float,split:split:int,species:meta:str
0,CCCCTAAGTC,0,0,Homo_sapiens
1,CTATATACTA,1,0,Homo_sapiens
2,TATATAGGTA,1,0,Homo_sapiens
3,GAGTGTGACT,0,1,Homo_sapiens
4,ATGAGATGGC,0,0,Homo_sapiens


In [110]:
# check how many positives there are per class split
df["split:split:int"].value_counts()

split:split:int
0    8
1    1
2    1
Name: count, dtype: int64

In [2]:
wrapper = PredictWrapper(model_class(), data_path, experiment, loss_dict, split = 1, batch_size = 1)
wrapper.predict()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x30 and 4980x1)